In [1]:
model_name = "my500_neptuneselected_meta"

In [2]:
import numpy as np
import pandas as pd
import gc
from time import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
timesheet = [time()]
def timer(statement):
    global timesheet
    timesheet.append(time())
    print statement+" :", (timesheet[-1]-timesheet[-2]),"seconds"
timer("Init...")

Init... : 0.000196933746338 seconds


In [4]:
def load_meta(directory, prefix, modeltype):
    m_tr = pd.read_csv(directory+prefix+"_train.csv")
    m_te = pd.read_csv(directory+prefix+"_test.csv")
    data = pd.concat([m_tr, m_te], axis=0).reset_index(drop=True)
    data.columns = ["{}_{}_{}".format(c, prefix, modeltype) if c!="SK_ID_CURR" else c for c in data.columns]
    return data

def join_features(data, features):
    for item in features:
        data = data.merge(item, how = "left", on = "SK_ID_CURR")
    return data

def load_data(datafile):
    global important_columns
    colnames = [c.replace(" ","_") for c in pd.read_csv(datafile, nrows= 1).columns]
    intersection = list(set(colnames).intersection(set(important_columns)))+["SK_ID_CURR"]

    df =  pd.read_csv(datafile, names = colnames, usecols = intersection, skiprows=1).dropna(axis=1, how="all")
    timer("Loaded {} with shape {} in ".format(datafile.split("/")[-1], df.shape))
    return df

def load_neptune(directory, modelname):
    m_tr = pd.read_csv(directory+"lightGBM_out_of_fold_train_predictions.csv", usecols = ["SK_ID_CURR","lightGBM_prediction"])
    m_te = pd.read_csv(directory+"lightGBM_out_of_fold_test_predictions.csv", usecols = ["SK_ID_CURR","lightGBM_prediction"])
    m_te = m_te.groupby("SK_ID_CURR")["lightGBM_prediction"].mean().reset_index()
    
    data = pd.concat([m_tr, m_te], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR", "neptune_{}".format(modelname)]
    return data

## Getting Feature Importance File

In [5]:
important_columns = pd.read_csv("../feature selector/importance/shap_importances.csv")
important_columns["feature"] = important_columns["feature"].apply(lambda x: x.replace(" ","_"))
important_columns = important_columns.loc[important_columns.shapely_mean > 0.0].reset_index(drop=True)
important_columns = important_columns.sort_values(by = "shapely_mean", ascending = False).reset_index(drop=True)
important_columns = important_columns.loc[0:500].feature.values.tolist()

## Loading Features

In [6]:
applications = load_data("../extractor/csv/application_features_V2.csv")
bureau_balance_bb = load_data("../extractor/csv/bureau_features_V1.csv")
credit_card_balance = load_data("../extractor/csv/credit_card_features_V1.csv")
installment_features = load_data("../extractor/csv/installment_features_V1.csv")
pos_cash_balance = load_data("../extractor/csv/pos_cash_features_V1.csv")
previous_apps = load_data("../extractor/csv/previous_application_features_V1.csv")

credit_card_balance_v3 = load_data("../extractor/csv/credit_card_features_V3.csv")
installment_features_v3 = load_data("../extractor/csv/installment_features_V3.csv")
pos_cash_balance_v3 = load_data("../extractor/csv/pos_cash_features_V3.csv")
previous_apps_v3 = load_data("../extractor/csv/previous_application_features_V3.csv")

base_features = [applications, bureau_balance_bb, credit_card_balance, installment_features, pos_cash_balance,previous_apps, credit_card_balance_v3,installment_features_v3,pos_cash_balance_v3, previous_apps_v3]

Loaded application_features_V2.csv with shape (356255, 75) in  : 9.01802706718 seconds
Loaded bureau_features_V1.csv with shape (305811, 84) in  : 10.341894865 seconds
Loaded credit_card_features_V1.csv with shape (103558, 16) in  : 10.4254870415 seconds
Loaded installment_features_V1.csv with shape (339587, 100) in  : 33.3528411388 seconds
Loaded pos_cash_features_V1.csv with shape (337252, 48) in  : 20.9478387833 seconds
Loaded previous_application_features_V1.csv with shape (338857, 93) in  : 22.0120661259 seconds
Loaded credit_card_features_V3.csv with shape (103558, 6) in  : 16.8840138912 seconds
Loaded installment_features_V3.csv with shape (339587, 69) in  : 27.695759058 seconds
Loaded pos_cash_features_V3.csv with shape (337252, 11) in  : 6.98300004005 seconds
Loaded previous_application_features_V3.csv with shape (338857, 9) in  : 8.40434503555 seconds


In [7]:
neptune_features = [
    load_neptune("../base neptune/m1/csv/", "m1"),
    load_neptune("../base neptune/m2/csv/", "m2"),
    load_neptune("../base neptune/m3/csv/", "m3"),
    load_neptune("../base neptune/m4/csv/", "m4"),
]

## Joining All Files

In [8]:
train = pd.read_csv("../data/application_train.csv", usecols = ["SK_ID_CURR","TARGET"])
test = pd.read_csv("../data/application_test.csv", usecols = ["SK_ID_CURR"])

neptune_train = pd.read_csv("../neptune extractor/data/train.csv")
neptune_test = pd.read_csv("../neptune extractor/data/test.csv")

train = pd.concat([train, neptune_train], axis=1)
test = pd.concat([test, neptune_test], axis=1)

del neptune_train, neptune_test
gc.collect()

data = pd.concat([train, test], axis=0).reset_index(drop=True)
data = join_features(data, base_features)
data = join_features(data, neptune_features)
del base_features,applications, bureau_balance_bb, credit_card_balance, installment_features, pos_cash_balance,previous_apps, credit_card_balance_v3,installment_features_v3,pos_cash_balance_v3, previous_apps_v3
gc.collect()

train = data.loc[data.TARGET.notnull()].reset_index(drop=True)
test = data.loc[data.TARGET.isnull()].reset_index(drop=True)

train_id = train[["SK_ID_CURR"]]
test_id = test[["SK_ID_CURR"]]
test_id_rank = test[["SK_ID_CURR"]]
target =train.TARGET

train.drop(["SK_ID_CURR", "TARGET"], axis=1, inplace=True)
test.drop(["SK_ID_CURR","TARGET"], axis=1, inplace = True)

# Defining Model

In [11]:
import lightgbm as lgb
def model_tree(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id):
    dtrain = lgb.Dataset(x_train, label=y_train)
    dval = lgb.Dataset(x_test, label=y_test)
    params = {
        'num_leaves': 33, #32
        'reg_alpha': 0.0, 
        'colsample_bytree': 0.05, 
        'subsample_freq': 1, 
        'learning_rate': 0.02, # 0.02
        'boosting_type': 'gbdt', 
        'nthread': 8, 
        'min_split_gain': 0.5, 
        'n_estimators': 10000, 
        'subsample': 1, 
        'reg_lambda': 100, 
        'objective': "binary", 
        'min_child_samples': 70, 
        'max_depth': -1,
        'class_weight': None,
        "bagging_seed" : 3143,
        "seed":1343,
        "metric":"auc",
        "is_unbalance": False,
        "scale_pos_weight": 1,
        "max_bin":300
    }
    model = lgb.train(params, dtrain, num_boost_round=5000,valid_sets=[dtrain, dval], early_stopping_rounds=200, verbose_eval=100)
    meta_train[test_index] = model.predict(x_test, num_iteration=model.best_iteration or 5000)
    meta_test.append(model.predict(test, num_iteration=model.best_iteration or 5000))
    
    global fold_roc
    fold_roc.append(roc_auc_score(y_test, meta_train[test_index]))
    # Calculate Feature Importance
    global feature_importance
    gain = model.feature_importance('gain')
    fold_feature_importance = pd.DataFrame({'feature':model.feature_name(), 'split':model.feature_importance('split'), 'gain':100 * gain / gain.sum()})
    feature_importance = feature_importance.append(fold_feature_importance, ignore_index=True) 

# Training Model

In [12]:
meta_train = np.zeros(train.shape[0])
meta_test = []
feature_importance = pd.DataFrame(columns = ["feature","split","gain"])
fold_roc = []

kf = StratifiedKFold(n_splits= 10, shuffle=True, random_state=47)
for fold_id, (train_index, test_index) in enumerate(kf.split(train, target)):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]

    model_tree(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id)

test_id["TARGET"] = np.array(meta_test).T.mean(axis=1)
test_id_rank["TARGET"] = pd.DataFrame(np.array(meta_test).T).rank(pct = True).mean(axis=1)
train_id["TARGET"] = meta_train

print "Overall ROC: {},  Mean ROC: {}, STD AUC: {}".format(roc_auc_score(target, meta_train), np.mean(fold_roc), np.std(fold_roc))

Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.777981	valid_1's auc: 0.765357
[200]	training's auc: 0.791706	valid_1's auc: 0.775022
[300]	training's auc: 0.802534	valid_1's auc: 0.781923
[400]	training's auc: 0.810259	valid_1's auc: 0.785686
[500]	training's auc: 0.81644	valid_1's auc: 0.787352
[600]	training's auc: 0.82189	valid_1's auc: 0.788664
[700]	training's auc: 0.826771	valid_1's auc: 0.789474
[800]	training's auc: 0.831356	valid_1's auc: 0.79005
[900]	training's auc: 0.835653	valid_1's auc: 0.790431
[1000]	training's auc: 0.839638	valid_1's auc: 0.790673
[1100]	training's auc: 0.843556	valid_1's auc: 0.790796
[1200]	training's auc: 0.847163	valid_1's auc: 0.790877
[1300]	training's auc: 0.850679	valid_1's auc: 0.790844
[1400]	training's auc: 0.853927	valid_1's auc: 0.790865
[1500]	training's auc: 0.857081	valid_1's auc: 0.790911
[1600]	training's auc: 0.860193	valid_1's auc: 0.790965
[1700]	training's auc: 0.863099	valid_1's auc: 0.7909

[1600]	training's auc: 0.860008	valid_1's auc: 0.790772
Early stopping, best iteration is:
[1450]	training's auc: 0.85533	valid_1's auc: 0.79089
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.777772	valid_1's auc: 0.768781
[200]	training's auc: 0.791491	valid_1's auc: 0.777163
[300]	training's auc: 0.802287	valid_1's auc: 0.783896
[400]	training's auc: 0.809973	valid_1's auc: 0.787797
[500]	training's auc: 0.816182	valid_1's auc: 0.789754
[600]	training's auc: 0.821696	valid_1's auc: 0.790988
[700]	training's auc: 0.826583	valid_1's auc: 0.791722
[800]	training's auc: 0.831264	valid_1's auc: 0.792258
[900]	training's auc: 0.835507	valid_1's auc: 0.792656
[1000]	training's auc: 0.839507	valid_1's auc: 0.793
[1100]	training's auc: 0.843362	valid_1's auc: 0.793151
[1200]	training's auc: 0.846918	valid_1's auc: 0.793253
[1300]	training's auc: 0.850514	valid_1's auc: 0.793351
[1400]	training's auc: 0.853857	valid_1's auc: 0.793421
[1500]	training's au

/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Overall ROC: 0.79636966673,  Mean ROC: 0.796417752998, STD AUC: 0.00546344808024


/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
fold_roc

[0.7909804784245346,
 0.8011806970337201,
 0.7988233461662906,
 0.8035471800054728,
 0.7944481279164768,
 0.7874717991470741,
 0.8039978121233317,
 0.7908902353632157,
 0.7934434280291995,
 0.7993944257718828]

In [14]:
train_id.to_csv("csv/{}_train.csv".format(model_name), index=False)
test_id.to_csv("csv/{}_test.csv".format(model_name), index=False)
test_id_rank.to_csv("csv/{}_rank_test.csv".format(model_name), index=False)

In [ ]:
-